DO NOT REMOVE THIS

This code was originally written by James Zhao (CSE167 WI23, A15939512)

In [2]:
import os, tqdm, shutil
from pathlib import Path
import numpy as np

In [3]:
def parse_pieces(pieces):
  out = []
  for piece in pieces:
    # vertex // normal
    if "//" in piece:
      out.append((int(piece[:piece.index("/")]), int(piece[piece.index("/") + 2:])))
    elif piece.count("/") == 2:
      v = piece.split("/")
      out.append((int(v[0]), int(v[1]), int(v[2])))
    else:
      out.append(int(piece))
  return out

def stats(verts):
  arr = np.array(verts)
  mins = np.min(arr, axis=0)
  maxs = np.max(arr, axis=0)
  return *mins, *maxs

def examine_materials(path):
  with open(path, "r") as f:
    lines = [l.strip() for l in f.readlines()]
    return set(list(filter(lambda x : x.startswith("usemtl"), lines)))

In [25]:
ALLOW = ["ambient", "emission", "diffuse", "specular", "normal"]
def parse(obj_path, out_path, name,
          eye = (10, 10, 10), center = (0, 10, 0), up = (0, 10, 0), fov = 90, 
          texture_folder = None, texture_prefix = None, materials = None, growth=1):
  # === Materials ===
  os.makedirs(out_path / name, exist_ok=True)
  textures_li = []
  materials_li = []
  material_idxs = []
  if texture_folder and texture_prefix and materials:
    print("=== Copying Textures ===")
    os.makedirs(out_path / name / "textures", exist_ok=True)
    for material_name in materials:
      materials_li.append(material_name)
      material_idx = []
      for key in ALLOW:
        if key in materials[material_name]:
          img = materials[material_name][key]
          material_idx.append(len(textures_li))
          textures_li.append(img)
          shutil.copy(texture_folder / img, out_path / name / "textures" / img)
        else:
          material_idx.append(-1)
      material_idxs.append(material_idx)
  
  with open(obj_path, "r") as f:
    obj = f.readlines()
  obj = [l.strip() for l in obj if not l.startswith("#") and not l.strip().isspace() and l.strip()]
  out = ["size 640 480", 
         f"camera {' '.join(map(str, eye))} {' '.join(map(str, center))} {' '.join(map(str, up))} {fov}", 
         "point 0 0 4 1 1 1", 
         "ambient 0 0 0", 
         "specular 0.2 0.2 0.2", 
         "diffuse 0.2 0.2 0.2", 
         "shininess 10"]
  # === PARSING ===
  verts = []
  norms = []
  uvs = []
  triangles = []
  trinorms = []
  tritexs = []
  cur_mat = 0
  print("===  Begin Parsing ===")
  for l in tqdm.tqdm(obj):
    vals = l.split()
    cmd = vals[0]
    if cmd == "usemtl":
      if vals[1] in materials_li:
        cur_mat = materials_li.index(vals[1])
      else:
        # raise Exception(f"Unaccounted material ({vals[1]} not in {materials_li})")
        pass
    elif cmd == "v":
      x, y, z = float(vals[1]), float(vals[2]), float(vals[3])
      verts.append((x, y, z))
    elif cmd == "vt":
      u, v = float(vals[1]), float(vals[2])
      uvs.append((u, v))
    elif cmd == "vn":
      x, y, z = float(vals[1]), float(vals[2]), float(vals[3])
      norms.append((x, y, z))
    elif cmd == "f":
      pieces = parse_pieces(vals[1:])
      if len(pieces) == 3 and type(pieces[0]) == int:
        triangles.append((pieces[0] - 1, pieces[1] - 1, pieces[2] - 1))
      elif len(pieces) == 3 and len(pieces[0]) == 2:
        trinorms.append((pieces[0][0] - 1, pieces[1][0] - 1, pieces[2][0] - 1, 
                         pieces[0][1] - 1, pieces[1][1] - 1, pieces[2][1] - 1))
      elif len(pieces) == 3 and len(pieces[0]) == 3:
        tritexs.append((pieces[0][0] - 1, pieces[1][0] - 1, pieces[2][0] - 1, 
                         pieces[0][2] - 1, pieces[1][2] - 1, pieces[2][2] - 1,
                         pieces[0][1] - 1, pieces[1][1] - 1, pieces[2][1] - 1,
                         cur_mat))
      elif len(pieces) == 4 and len(pieces[0]) == 3:
        tritexs.append((pieces[0][0] - 1, pieces[1][0] - 1, pieces[2][0] - 1, 
                         pieces[0][2] - 1, pieces[1][2] - 1, pieces[2][2] - 1,
                         pieces[0][1] - 1, pieces[1][1] - 1, pieces[2][1] - 1,
                         cur_mat))
        tritexs.append((pieces[0][0] - 1, pieces[2][0] - 1, pieces[3][0] - 1, 
                  pieces[0][2] - 1, pieces[2][2] - 1, pieces[3][2] - 1,
                  pieces[0][1] - 1, pieces[2][1] - 1, pieces[3][1] - 1,
                  cur_mat))
    else:
      print(f"Unrecognized Command: {cmd} {vals[1:]}")
  
  print("=== Finished Parsing ===")
  minx, miny, minz, maxx, maxy, maxz = stats(verts)
  meanx, meany, meanz = (minx + maxx) / 2, (miny + maxy) / 2, (minz + maxz) / 2
  rngx, rngy, rngz = (maxx - minx) / 2, (maxy - miny) / 2, (maxz - minz) / 2
  rng = max(rngx, rngy, rngz)
  print(f"Vertices: {len(verts)}, Normals: {len(norms)}, Uvs: {len(uvs)}")
  print(f"Triangles: {len(triangles)}, Trinorms: {len(trinorms)}, Tritexs: {len(tritexs)}")
  print(f"Bounds: ({minx},{maxx}), ({miny},{maxy}]),({minz},{maxz})")
  print(f"Centers: ({meanx},{meany},{meanz})")
  print(f"Ranges: ({rngx},{rngy},{rngz})")
  verts = np.array(verts)
  norms = np.array(norms)
  verts = (verts - np.array([[meanx, meany, meanz]])) / rng * growth
  print("Mean after centralizing", np.mean(verts, axis=0))
  print("Mins/Maxs after centralizing", np.min(verts, axis=0), np.max(verts, axis=0))
  for texture in textures_li:
    out.append(f"texture {Path(texture_prefix) / texture}")
  for idx, (x, y, z) in enumerate(verts):
    out.append(f"vertex {x:0.12f} {y:0.12f} {z:0.12f}")
  for nx, ny, nz in norms:
    out.append(f"normal {nx} {ny} {nz}")
  for u, v in uvs:
    out.append(f"texvert {u} {v}")
  for v0, v1, v2 in triangles:
    out.append(f"tri {v0} {v1} {v2}")
  for v0, v1, v2, n0, n1, n2 in trinorms:
    out.append(f"cooler_trinorm {v0} {v1} {v2} {n0} {n1} {n2}")
  prev_matid = None
  for idx, (v0, v1, v2, n0, n1, n2, uv0, uv1, uv2, matid) in enumerate(tritexs):
    if prev_matid is None or prev_matid != matid:
      mat_triple = material_idxs[matid]
      prev_matid = matid
      out.append(f"texturemaps {' '.join([str(v) for v in mat_triple])} 0.2 1.0 1.0 1.0")
    # If any daring adventurer ever wants to use this code, just swap out these
    # appends with something lke out.append(f"tri {v0} {v1} {v2}")
    out.append(f"tritex {v0} {v1} {v2} {n0} {n1} {n2} {uv0} {uv1} {uv2}")
  print("=== Writing to File ===")
  with open(out_path / name / f"{name}.test", "w") as f:
    f.write(f"# Output: {out_path}\n")
    for line in tqdm.tqdm(out):
      f.write(f"{line}\n")

# Thunderous Pulse

In [18]:
load_path = Path("models/src/bow/source/bow1.obj")
out_path = Path("models/out")
texture_prefix = Path("../testscenes/bow/textures")
texture_folder = Path("models/src/bow/textures")
name = "bow"
print(examine_materials(load_path))
materials = {
  "BASE": {
    "ambient": "bow1_BASE_BaseColor.png",
    "emission": "bow1_BASE_Emissive.png"
  },
  "skin": {
    "ambient": "bow1_skin_BaseColor.png",
    "emission": "bow1_skin_Emissive.png"
  }
}

{'usemtl skin', 'usemtl BASE'}


In [38]:
parse(load_path, out_path, name,
      eye=(0, 0, 2), center = (0, 0, 0), up= (0, 1, 0), 
      texture_prefix = texture_prefix, texture_folder = texture_folder,
      materials = materials, growth=2)

=== Copying Textures ===
===  Begin Parsing ===


100%|██████████| 48323/48323 [00:00<00:00, 302019.04it/s]

Unrecognized Command: mtllib ['bow1.mtl']
Unrecognized Command: g ['Plane']
Unrecognized Command: g ['Circle_004']
Unrecognized Command: g ['Cylinder']
Unrecognized Command: g ['Sphere_001']
Unrecognized Command: g ['Line']
Unrecognized Command: g ['Cylinder_001']
Unrecognized Command: g ['Cylinder_002']
Unrecognized Command: g ['Plane_001']
Unrecognized Command: g ['Plane_002']
Unrecognized Command: g ['Plane_004']
Unrecognized Command: g ['Cylinder_003']
Unrecognized Command: g ['Cylinder_006']
Unrecognized Command: g ['BezierCurve_001']
Unrecognized Command: g ['Cylinder_004']
Unrecognized Command: g ['Cylinder_005']
Unrecognized Command: g ['BezierCurve']
Unrecognized Command: g ['BezierCurve_004']
Unrecognized Command: g ['BezierCurve_003']
Unrecognized Command: g ['BezierCurve_005']
=== Finished Parsing ===
Vertices: 8689, Normals: 8689, Uvs: 13594
Triangles: 0, Trinorms: 0, Tritexs: 17312
Bounds: (-124.941194,61.057006), (-230.515338,438.719654]),(-8.043065,10.150165)
Centers: (

=== Writing to File ===


100%|██████████| 48297/48297 [00:00<00:00, 1097661.33it/s]


# ICHIGO

In [55]:
load_path = Path("models/src/strawberry/Strawberry_obj.obj")
out_path = Path("models/out")
texture_prefix = Path("../testscenes/strawberry/textures")
texture_folder = Path("models/src/strawberry/Texture")
name = "strawberry"
parse(load_path, out_path, name,
      eye=(0, 1, 2), center = (0, 0, 0), up= (0, 1, 0), 
      texture_prefix = texture_prefix, texture_folder = texture_folder)

=== Copying Textures ===
===  Begin Parsing ===


 68%|██████▊   | 197092/289125 [00:00<00:00, 666860.24it/s]

Unrecognized Command: mtllib ['Strawberry_obj.mtl']
Unrecognized Command: o ['strawberry']
Unrecognized Command: s ['1']


100%|██████████| 289125/289125 [00:00<00:00, 345843.41it/s]


=== Finished Parsing ===
Vertices: 65330, Normals: 65314, Uvs: 93129
Triangles: 0, Trinorms: 0, Tritexs: 129702
Bounds: (-2.431606,2.470415), (0.017322,7.101389]),(-2.520095,2.380674)
Centers: (0.019404500000000047,3.5593555,-0.06971050000000001)
Ranges: (2.4510104999999998,3.5420335,2.4503845)
BIG range: (3.5420335)
Mean/std verts after centralizing [-0.00031577  0.03935662  0.00151949] [0.36166856 0.55393257 0.3614472 ]
=== Writing to File ===


100%|██████████| 548515/548515 [00:00<00:00, 1202882.28it/s]


# AMOS BOW

# HARDEDGE

In [23]:
# https://sketchfab.com/3d-models/10-genshin-impact-inspired-sword-3d-models-a25cde4706d04e4cb5ea7ff89ceb5dd9
name = "hardedge"
load_path = Path(f"models/src/{name}/source/Hardedge_LP.obj")
out_path = Path("models/out")
texture_prefix = Path(f"../testscenes/{name}/textures")
texture_folder = Path(f"models/src/{name}/textures")

for v in examine_materials(load_path):
  print(v)
materials = {
  "Hardedge": {
    "ambient": "T_Hardedge_Cleaner_BaseColor.tga.png",
    "normal": "T_Hardedge_Cleaner_Normal.tga.png"
  },
}

usemtl Hardedge


In [28]:
parse(load_path, out_path, name,
      eye=(0, 2, 0), center = (0, 0, 0), up= (0, 0, 1), 
      texture_prefix = texture_prefix, texture_folder = texture_folder,
      materials = materials, growth=2)

=== Copying Textures ===
===  Begin Parsing ===


100%|██████████| 45302/45302 [00:00<00:00, 482677.55it/s]

Unrecognized Command: mtllib ['hardedge_lp.mtl']
Unrecognized Command: g ['blade_low']
Unrecognized Command: g ['bladesupport_low']
Unrecognized Command: g ['handguard_low']
Unrecognized Command: g ['materiaball_low_02']
Unrecognized Command: g ['materiaball_low_03']
Unrecognized Command: g ['materiaball_low_01']
Unrecognized Command: g ['handlewrap_low']
Unrecognized Command: g ['handleextension_low']
Unrecognized Command: g ['pommel_low']
Unrecognized Command: g ['materiatube_low_01']
Unrecognized Command: g ['materiatube_low_03']
Unrecognized Command: g ['materiatube_low_02']
Unrecognized Command: g ['nails_low']
=== Finished Parsing ===
Vertices: 9023, Normals: 9023, Uvs: 10230
Triangles: 0, Trinorms: 0, Tritexs: 16999
Bounds: (-118.265324,66.246414), (-2.539807,2.572501]),(-23.651881,17.636719)
Centers: (-26.009455000000003,0.01634699999999989,-3.007581)
Ranges: (92.255869,2.5561540000000003,20.6443)
Mean after centralizing [-0.06884359  0.0006732   0.02725188]
Mins/Maxs after cen


100%|██████████| 45285/45285 [00:00<00:00, 1525443.38it/s]
